# Setup

First we load in our datasets for analysis. This includes our cluster datasets for both stages, where each row is a cluster as well as protein datasets where each row is a protein.

In [785]:
import pandas as pd
# load in our cluster data
s3_df = pd.read_csv("data/generated_tables/s3_network.tsv", sep="\t")
s5_df = pd.read_csv("data/generated_tables/s5_network.tsv", sep="\t")
# keep track of the stage
s5_df["stage"] = 5
s3_df["stage"] = 3


# add essentiality data onto clusters
s5_essentiality_df = pd.read_csv("data/generated_tables/s5_essentiality_df.tsv", sep="\t")
s3_essentiality_df = pd.read_csv("data/generated_tables/s3_essentiality_df.tsv", sep="\t")
# keep track of stage
s5_essentiality_df["stage"] = 5
s3_essentiality_df["stage"] = 3

# combine the two dataframes
protein_concat_df = pd.concat([s5_essentiality_df, s3_essentiality_df])

In [786]:
# Appending redundant tag onto essentiality data
s3_essentiality_df["redundant"] = s3_essentiality_df["Product Description"].duplicated(keep=False)
s5_essentiality_df["redundant"] = s5_essentiality_df["Product Description"].duplicated(keep=False)
for i, row in s3_essentiality_df.iterrows():
    if "unknown function" in row["Product Description"]:
        s3_essentiality_df.at[i, "redundant"] = False
for i, row in s5_essentiality_df.iterrows():
    if "unknown function" in row["Product Description"]:
        s5_essentiality_df.at[i, "redundant"] = False

# add column for essentiality threshold
s3_essentiality_df["under_MIS_threshold"] = s3_essentiality_df["essential"]
s5_essentiality_df["under_MIS_threshold"] = s5_essentiality_df["essential"]

# change essential to be true if either redundant or under threshold
s3_essentiality_df["essential"] = s3_essentiality_df["under_MIS_threshold"] | s3_essentiality_df["redundant"]
s5_essentiality_df["essential"] = s5_essentiality_df["under_MIS_threshold"] | s5_essentiality_df["redundant"]

s3_essentiality_df.drop("Unnamed: 0", axis=1, inplace=True)
s5_essentiality_df.drop("Unnamed: 0", axis=1, inplace=True)

s3_essentiality_df.to_csv("data/generated_tables/updated_s3_essentiality_df.tsv", sep="\t", index=False)
s5_essentiality_df.to_csv("data/generated_tables/updated_s5_essentiality_df.tsv", sep="\t", index=False)

s3_essentiality_df[s3_essentiality_df["Accession ID"] == "Q8IIV1"]

,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
156,Q8IIV1,PF3D7_1105100,histone H2B,NaN,NaN,"['Q8IB24', 'C6KSV0', 'Q8IIV2', 'C6KT18', 'K7NT...",False,9,0.000998,0.289366,3,False,False


In [787]:
print("Number of clusters in s5 network greater than random: ", sum(s5_df["significant"]))
print("Number of clusters in s3 network greater than random: ", sum(s3_df["significant"]))

Number of clusters in s5 network greater than random:  64
Number of clusters in s3 network greater than random:  30


In [788]:
from ast import literal_eval

# function to convert str representation of Proteins cluster column in df to sets
def convert_data(df):
    df["Proteins"] = df["Proteins"].apply(literal_eval)
    return df 

print("data type of cluster before converting: ", type(s3_df["Proteins"][0]))
for df in [s3_df, s5_df]:
    df = convert_data(df)
print("data type of cluster after converting: ", type(s3_df["Proteins"][0]))


data type of cluster before converting:  <class 'str'>
data type of cluster after converting:  <class 'set'>


## Analyzing Network Stats

In [789]:
# 
s5_essentiality_df[s5_essentiality_df["essential"]].sort_values("Degree", ascending=False)

,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
114,C0H5H0,PF3D7_1344200,endoplasmic reticulum chaperone GRP170,0.143,-2.839,"['O97282', 'Q8II43', 'Q8IFP3', 'Q8ILP6', 'C6KS...",True,69,0.148560,0.445327,5,False,True
109,Q8IAX5,PF3D7_0813900,"40S ribosomal protein S16, putative",0.141,-3.879,"['Q8IE10', 'C6KT19', 'C6KT25', 'Q8IIB4', 'Q8IE...",True,58,0.033885,0.426100,5,False,True
41,Q8I323,PF3D7_0912900,"26S proteasome regulatory subunit RPN8, putative",0.122,-3.009,"['Q8ILE3', 'Q76NM6', 'Q8IKB2', 'Q8I0V2', 'Q8I2...",True,53,0.053808,0.421672,5,False,True
62,Q8IDR9,PF3D7_1342000,40S ribosomal protein S6,0.127,-2.986,"['Q8ILI2', 'Q8IIA2', 'Q8IK15', 'Q8IIV2', 'A0A1...",True,53,0.022999,0.415909,5,False,True
85,C0H5C2,PF3D7_1317800,40S ribosomal protein S19,0.133,-2.762,"['A0A144A3N9', 'Q8IIB4', 'A0A5K1K8V8', 'Q8IEK3...",True,52,0.020566,0.411225,5,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Q8IM52,PF3D7_1404200,"conserved Plasmodium protein, unknown function",0.992,-1.804,['Q8IC43'],True,1,0.000000,0.255541,5,False,True
147,Q8IL86,PF3D7_1438100,"translocation protein SEC62, putative",0.169,-2.953,['Q8IEC8'],True,1,0.000000,0.180152,5,False,True
38,Q8IIB6,PF3D7_1124700,"GrpE protein homolog, mitochondrial, putative",0.122,-2.922,['Q76NN6'],True,1,0.000000,0.265770,5,False,True
130,Q8IAW0,PF3D7_0812400,karyopherin alpha,0.153,-2.927,['Q76NN6'],True,1,0.000000,0.265770,5,False,True


## Tagging Protein Essentiality to Clusters

In [790]:
# tagging protein essentiality to clusters
def calculate_cluster_essentiality(cluster):
    total = 0
    for protein in cluster:
        if protein in s5_essentiality_df["Accession ID"].values:
            total += s5_essentiality_df[s5_essentiality_df["Accession ID"] == protein]["essential"].values[0]
    return total/len(cluster)

lst = []
for i, row in s5_df.iterrows():
    lst.append(calculate_cluster_essentiality(row["Proteins"]))
s5_df["essentiality"] = lst

lst = []
for i, row in s3_df.iterrows():
    lst.append(calculate_cluster_essentiality(row["Proteins"]))
s3_df["essentiality"] = lst

## Analyzing Between Clusters

In [791]:
# # combine the two dataframes
# cluster_concat_df = pd.concat([s3_df, s5_df])


# rename columns
s3_df.rename(columns={"Number": "s3_number",
                      "Proteins": "s3_proteins",
                      "avg_spearman": "s3_avg_spearman",
                      "size": "s3_size",
                      "essentiality": "s3_essentiality"}, inplace=True),

s5_df.rename(columns={"Number": "s5_number",
                      "Proteins": "s5_proteins",
                      "avg_spearman": "s5_avg_spearman",
                      "size": "s5_size",
                      "essentiality": "s5_essentiality"}, inplace=True)

In [792]:
# cross join the two dataframes so each row is a combination of s3 and s5 clusters
crossed_df = pd.merge(
    s3_df[["s3_number", "s3_proteins", "s3_avg_spearman", "s3_size", "s3_essentiality"]],
    s5_df[['s5_number', 's5_proteins', 's5_avg_spearman', 's5_size', "s5_essentiality"]],
    how="cross")

# reordering the columns
crossed_df = crossed_df.loc[:, ["s3_number", "s5_number", "s3_proteins", "s5_proteins", 
                                "s3_avg_spearman", "s5_avg_spearman", "s3_size", "s5_size"]]
crossed_df["total_size"] = crossed_df["s3_size"] + crossed_df["s5_size"]

In [793]:
# look for common proteins across clusters
common_proteins = []
for _, row in crossed_df.iterrows():
    row_common_proteins = []
    for protein in row["s3_proteins"]:
        if protein in row["s5_proteins"]:
            row_common_proteins.append(protein)
    common_proteins.append(row_common_proteins)
crossed_df["common_proteins"] = common_proteins

In [794]:
# look for common proteins across clusters
def get_common_proteins(proteins1, proteins2):
    common_proteins = []
    # iterate over crossed df
    for _, row in crossed_df.iterrows():
        row_common_proteins = []
        # if protein in s3 is in s5, add to list of common proteins
        for protein in row[proteins1]:
            if protein in row[proteins2]:
                row_common_proteins.append(protein)
        common_proteins.append(row_common_proteins)
    return common_proteins

In [795]:
# add number of overlapping proteins
crossed_df["num_overlap"] = crossed_df["common_proteins"].apply(len)
# sort by number of overlaps
filtered_crossed_df = crossed_df[crossed_df["num_overlap"] > 0]
filtered_crossed_df.sort_values("num_overlap", ascending=False).head(5)

,s3_number,s5_number,s3_proteins,s5_proteins,s3_avg_spearman,s5_avg_spearman,s3_size,s5_size,total_size,common_proteins,num_overlap
509,6,5,"{Q8IC01, C6KT34, Q8IDG2, Q8I0U7, Q8IDG3, Q6ZMA...","{Q8IJA0, Q8IKC9, C6KT34, Q8IDG2, O77323, Q8IBR...",0.721655,0.800470,24,35,59,"[C6KT34, Q8IDG2, Q8I0U7, Q8IDG3, Q8II43, A0A5K...",15
89,1,5,"{Q8IE84, Q8IC01, C6KT34, Q8IDG2, Q8I0U7, Q8IDG...","{Q8IJA0, Q8IKC9, C6KT34, Q8IDG2, O77323, Q8IBR...",0.738465,0.800470,24,35,59,"[C6KT34, Q8IDG2, Q8I0U7, Q8IDG3, Q8II43, A0A5K...",15
2,0,2,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.376935,0.516712,16,65,81,"[Q8IET7, Q8IBV7, Q8I0P6, K7NTP5, Q8IB24, Q8I0V...",11
1178,14,2,"{Q8IBV7, Q8IAX8, C6KSV0, Q8I608, K7NTP5, Q8IK8...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.331886,0.516712,14,65,79,"[Q8IBV7, Q8IAX8, K7NTP5, Q8IB24, Q8IJX8, Q8I0V...",10
1514,18,2,"{Q8IET7, Q8I0P6, O96252, O97247, Q8IK89, K7NTP...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.426576,0.516712,21,65,86,"[Q8IET7, Q8I0P6, K7NTP5, O97285, Q8IKR1, Q8I0V...",7


In [796]:
def calculate_overlap(row, size_col1, size_col2):
    return row["num_overlap"] / min(row[size_col1], row[size_col2])

def calculate_jaccard_index(row, proteins_col1, proteins_col2):
    return len(row["common_proteins"]) / len(set(list(row[proteins_col1]) + list(row[proteins_col2])))

crossed_df["percent_overlap"] = crossed_df.apply(calculate_overlap, args=["s3_size", "s5_size"], axis=1)
crossed_df["jaccard_index"] = crossed_df.apply(calculate_jaccard_index, args=["s3_proteins", "s5_proteins"], axis=1)
crossed_df["similarity_score"] = (crossed_df["percent_overlap"] + crossed_df["jaccard_index"]) / 2

In [797]:
crossed_df[["s3_number", "s5_number", "s3_avg_spearman", "s5_avg_spearman", "s3_size", "s5_size", "total_size", "num_overlap", "jaccard_index", "percent_overlap", "similarity_score"]].sort_values("similarity_score", ascending=False).head(5)

,s3_number,s5_number,s3_avg_spearman,s5_avg_spearman,s3_size,s5_size,total_size,num_overlap,jaccard_index,percent_overlap,similarity_score
1848,22,0,0.506917,0.500198,2,6,8,2,0.333333,1.000,0.666667
1466,17,38,0.486660,0.639987,4,6,10,3,0.428571,0.750,0.589286
996,11,72,0.704249,0.599209,5,5,10,3,0.428571,0.600,0.514286
946,11,22,0.704249,0.379117,5,6,11,3,0.375000,0.600,0.487500
509,6,5,0.721655,0.800470,24,35,59,15,0.340909,0.625,0.482955


In [798]:
# top 10 clusters with highest percent common
crossed_df[crossed_df["num_overlap"] > 2].sort_values("percent_overlap", ascending=False).head(5)

,s3_number,s5_number,s3_proteins,s5_proteins,s3_avg_spearman,s5_avg_spearman,s3_size,s5_size,total_size,common_proteins,num_overlap,percent_overlap,jaccard_index,similarity_score
1466,17,38,"{Q8IHR8, Q76NM4, A0A5K1K8H7, Q8I3W9}","{Q8IIK8, A0A5K1K8H7, Q8I3W9, Q8I274, Q76NM4, C...",0.486660,0.639987,4,6,10,"[Q76NM4, A0A5K1K8H7, Q8I3W9]",3,0.750000,0.428571,0.589286
1178,14,2,"{Q8IBV7, Q8IAX8, C6KSV0, Q8I608, K7NTP5, Q8IK8...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.331886,0.516712,14,65,79,"[Q8IBV7, Q8IAX8, K7NTP5, Q8IB24, Q8IJX8, Q8I0V...",10,0.714286,0.144928,0.429607
87,1,3,"{Q8IE84, Q8IC01, C6KT34, Q8IDG2, Q8I0U7, Q8IDG...","{Q8IE84, Q76NM6, Q8IEP9, Q8I2H3, Q8IDS0, Q6ZMA...",0.738465,0.734660,24,7,31,"[Q8IE84, Q6ZMA8, Q8I280, Q8I2H3, Q76NM6]",5,0.714286,0.192308,0.453297
2,0,2,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.376935,0.516712,16,65,81,"[Q8IET7, Q8IBV7, Q8I0P6, K7NTP5, Q8IB24, Q8I0V...",11,0.687500,0.157143,0.422321
2270,27,2,"{Q8IAX8, C6KSV0, Q8IBV7, Q8IJX3, Q8IIV2, C6KT1...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.349638,0.516712,9,65,74,"[Q8IAX8, Q8IBV7, Q8IIV2, C6KT18, Q8I5H4, Q8IIV1]",6,0.666667,0.088235,0.377451


In [799]:
# top 10 clusters with lowest percent common
crossed_df.sort_values(["percent_overlap", "total_size"], ascending=[True, False]).head(5)

,s3_number,s5_number,s3_proteins,s5_proteins,s3_avg_spearman,s5_avg_spearman,s3_size,s5_size,total_size,common_proteins,num_overlap,percent_overlap,jaccard_index,similarity_score
590,7,2,"{Q8IAV1, C0H571, Q8IKC8, Q8I484, Q8IEU2, Q8IBN...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.552264,0.516712,11,65,76,[],0,0.0,0.0,0.0
1346,16,2,"{Q8IC01, Q8I2X4, Q8IDG2, Q8II24, Q8IEK1, O9722...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.626482,0.516712,10,65,75,[],0,0.0,0.0,0.0
2354,28,2,"{Q8IC01, Q7K6A5, Q8IEK1, O97227, Q8I2J3, Q8II3...","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.438453,0.516712,7,65,72,[],0,0.0,0.0,0.0
1262,15,2,"{Q8I490, Q8IFM0, Q8I206, Q8I2F4, Q8IIX5, Q8IDG9}","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.384848,0.516712,6,65,71,[],0,0.0,0.0,0.0
758,9,2,"{Q8I0X1, Q8ILB6, P61074, Q8IEJ6, O97227}","{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.678557,0.516712,5,65,70,[],0,0.0,0.0,0.0


In [800]:
# get clusters with no common proteins across all clusters
x = crossed_df[crossed_df["s3_number"] == 1]
y = x["num_overlap"] > 1
bool(y.sum() == 0)

False

In [801]:
lst = []
for i in s3_df["s3_number"]:
    x = crossed_df[crossed_df["s3_number"] == i]
    y = x["num_overlap"] > 1
    lst.append(bool(y.sum() == 0))
s3_df["unique between stages"] = lst

lst = []
for i in s5_df["s5_number"]:
    x = crossed_df[crossed_df["s5_number"] == i]
    y = x["num_overlap"] > 1
    lst.append(bool(y.sum() == 0))
s5_df["unique between stages"] = lst

In [802]:
s5_df[s5_df["unique between stages"]].sort_values("s5_size", ascending=False).head(5)

,s5_number,s5_proteins,s5_avg_spearman,s5_size,avg_spearman_random,significant,large,stage,s5_essentiality,unique between stages
17,17,"{Q8IKT2, Q8III8, P50250, Q76NM3, Q8I3X4, Q8I5B...",0.489377,9,0.388635,True,True,5,0.777778,True
23,23,"{Q8I0V4, A0A144A1R5, Q8IIX0, Q8IJ34, O97249, Q...",0.496267,9,0.406262,True,True,5,0.666667,True
15,15,"{Q8IEQ1, Q8IAR6, Q8IKH3, Q8IJM0, Q8ILE3, Q8IC0...",0.758082,8,0.392335,True,True,5,0.625000,True
63,63,"{Q8IKT2, P50250, Q8I6U7, Q8I3X4, Q8IIR8, P2736...",0.499435,8,0.392750,True,True,5,0.625000,True
30,30,"{C0H4C7, Q8IBR6, Q8I3Y6, Q8ILS7, Q8IBZ4, Q8II8...",0.879400,7,0.397545,True,True,5,0.428571,True


In [803]:
s5_df[s5_df["unique between stages"] & (s5_df["s5_size"] > 2)].sort_values(["s5_avg_spearman", "s5_size"], ascending=False).head(5)

,s5_number,s5_proteins,s5_avg_spearman,s5_size,avg_spearman_random,significant,large,stage,s5_essentiality,unique between stages
66,66,"{Q8II42, Q8I3A1, P61074, Q8II92, C6S3I6}",0.940508,5,0.399015,True,True,5,0.600000,True
30,30,"{C0H4C7, Q8IBR6, Q8I3Y6, Q8ILS7, Q8IBZ4, Q8II8...",0.879400,7,0.397545,True,True,5,0.428571,True
1,1,"{Q8IIW2, Q8IBS3, Q8I246, Q8IIA4, Q8ILP6, Q8IDZ9}",0.873057,6,0.381010,True,True,5,1.000000,True
36,36,"{C6S3I6, Q8II42, Q8I3A1, Q8I5Q3}",0.838632,4,0.423284,True,True,5,0.500000,True
34,34,"{Q8IK83, Q8IKS0, Q8IM24, Q8IKW0}",0.814394,4,0.412509,True,True,5,1.000000,True


## Comparing Within Cluster

To compare PPIs within a cluster, we cross join the stage 3 dataset with itself,
and see if the same proteins appear in any 2 clusters. We call this attribute 'unique within stage'.

Additionally, we cross join them 

In [804]:
# TODO: 

# cross join the two dataframes so each row is a combination of s3 and s5 clusters
s3_crossed_df = pd.merge(
    s3_df[["s3_number", "s3_proteins", "s3_avg_spearman", "s3_size"]].rename(columns={"s3_number": "number_1",
                                                                                        "s3_proteins": "proteins_1",
                                                                                        "s3_avg_spearman": "avg_spearman_1",
                                                                                        "s3_size": "size_1"}),
    s3_df[["s3_number", "s3_proteins", "s3_avg_spearman", "s3_size"]].rename(columns={"s3_number": "number_2",
                                                                                        "s3_proteins": "proteins_2",
                                                                                        "s3_avg_spearman": "avg_spearman_2",
                                                                                        "s3_size": "size_2"}),
    how="cross")
# drop rows where the two clusters are the same
s3_crossed_df.drop(s3_crossed_df[s3_crossed_df["number_1"] == s3_crossed_df["number_2"]].index, inplace=True)
# calculating total size
s3_crossed_df["total_size"] = s3_crossed_df["size_1"] + s3_crossed_df["size_2"]
s3_crossed_df.head()

# look for common proteins across clusters
common_proteins = []
for _, row in s3_crossed_df.iterrows():
    row_common_proteins = []
    for protein in row["proteins_1"]:
        if protein in row["proteins_2"]:
            row_common_proteins.append(protein)
    common_proteins.append(row_common_proteins)
s3_crossed_df["common_proteins"] = common_proteins

# add number of overlapping proteins
s3_crossed_df["num_overlap"] = s3_crossed_df["common_proteins"].apply(len)
# sort by number of overlaps
filtered_crossed_df = crossed_df[crossed_df["num_overlap"] > 0]
filtered_crossed_df.sort_values("num_overlap", ascending=False).head(5)

lst = []
for i in s3_df["s3_number"]:
    x = s3_crossed_df[s3_crossed_df["number_1"] == i]
    y = x["num_overlap"] > 1
    lst.append(bool(y.sum() == 0))
s3_df["unique within stage"] = lst

lst = []
# loop over each s3 cluster
for i in s3_df["s3_number"]:
    # get list of cluster pairs for current s3 cluster
    x = crossed_df[crossed_df["s3_number"] == i]
    # see if there is any overlapping proteins
    y = x["num_overlap"] > 1
    # if there's no overlapping proteins across all pairs, then the proteins
    # in this cluster are unique between stages
    lst.append(bool(y.sum() == 0))
s3_df["unique between stages"] = lst

In [805]:
# TODO: 

# cross join the two dataframes so each row is a combination of s3 and s5 clusters
s5_crossed_df = pd.merge(
    s5_df[["s5_number", "s5_proteins", "s5_avg_spearman", "s5_size"]].rename(columns={"s5_number": "number_1",
                                                                                        "s5_proteins": "proteins_1",
                                                                                        "s5_avg_spearman": "avg_spearman_1",
                                                                                        "s5_size": "size_1"}),
    s5_df[["s5_number", "s5_proteins", "s5_avg_spearman", "s5_size"]].rename(columns={"s5_number": "number_2",
                                                                                        "s5_proteins": "proteins_2",
                                                                                        "s5_avg_spearman": "avg_spearman_2",
                                                                                        "s5_size": "size_2"}),
    how="cross")
# drop rows where the two clusters are the same
s5_crossed_df.drop(s5_crossed_df[s5_crossed_df["number_1"] == s5_crossed_df["number_2"]].index, inplace=True)
# calculating total size
s5_crossed_df["total_size"] = s5_crossed_df["size_1"] + s5_crossed_df["size_2"]

# look for common proteins across clusters
common_proteins = []
for _, row in s5_crossed_df.iterrows():
    row_common_proteins = []
    for protein in row["proteins_1"]:
        if protein in row["proteins_2"]:
            row_common_proteins.append(protein)
    common_proteins.append(row_common_proteins)
s5_crossed_df["common_proteins"] = common_proteins

# add number of overlapping proteins
s5_crossed_df["num_overlap"] = s5_crossed_df["common_proteins"].apply(len)
# sort by number of overlaps
filtered_crossed_df = crossed_df[crossed_df["num_overlap"] > 0]
filtered_crossed_df.sort_values("num_overlap", ascending=False).head(5)

# get clusters with proteins unique WITHIN a stage
lst = []
for i in s5_df["s5_number"]:
    x = s5_crossed_df[s5_crossed_df["number_1"] == i]
    y = x["num_overlap"] > 1
    lst.append(bool(y.sum() == 0))
s5_df["unique within stage"] = lst

# get clusters with proteins unique BETWEEN stages
lst = []
for i in s5_df["s5_number"]:
    x = crossed_df[crossed_df["s5_number"] == i]
    y = x["num_overlap"] > 1
    lst.append(bool(y.sum() == 0))
s5_df["unique between stages"] = lst

In [806]:
unique_s5_clusters = s5_df[s5_df["unique within stage"] & 
      s5_df["unique between stages"] & 
      s5_df["large"] & 
      s5_df["significant"]].sort_values("s5_avg_spearman", ascending=False)
print(unique_s5_clusters.head(5))
unique_s5_clusters.to_csv("data/generated_tables/unique_s5_clusters.tsv", sep="\t", index=False)

    s5_number                                        s5_proteins  \
19         19               {Q8I1S0, A0A143ZY58, Q8IKH3, Q8IM66}   
31         31               {Q8IL48, C6KTA3, A0A5K1K967, C6KSV2}   
15         15  {Q8IEQ1, Q8IAR6, Q8IKH3, Q8IJM0, Q8ILE3, Q8IC0...   
32         32                   {C6S3G2, O96252, Q8I3N3, C6KT09}   
40         40                   {C0H4W2, Q8I3Q7, Q8IIT3, Q8I5V6}   

    s5_avg_spearman  s5_size  avg_spearman_random  significant  large  stage  \
19         0.796937        4             0.397808         True   True      5   
31         0.795290        4             0.392043         True   True      5   
15         0.758082        8             0.392335         True   True      5   
32         0.747036        4             0.393874         True   True      5   
40         0.701087        4             0.413668         True   True      5   

    s5_essentiality  unique between stages  unique within stage  
19            0.750                   True  

In [807]:
s3_df[s3_df["unique within stage"] & s3_df["unique between stages"] & s3_df["large"] & s3_df["significant"]]
#{Q8I3M5, C6KTB3, O77312}	

,s3_number,s3_proteins,s3_avg_spearman,s3_size,avg_spearman_random,significant,large,stage,s3_essentiality,unique between stages,unique within stage
13,13,"{Q8I6S5, Q8IJ34, Q8IEA6}",0.474638,3,0.372556,True,True,3,1.0,True,True
19,19,"{O77312, Q8I3M5, C6KTB3}",0.774704,3,0.386410,True,True,3,1.0,True,True


In [808]:
s5_df[s5_df["unique within stage"] & s5_df["unique between stages"] & s5_df["large"] & s5_df["significant"]].sort_values("s5_avg_spearman", ascending=False).head(5)

,s5_number,s5_proteins,s5_avg_spearman,s5_size,avg_spearman_random,significant,large,stage,s5_essentiality,unique between stages,unique within stage
19,19,"{Q8I1S0, A0A143ZY58, Q8IKH3, Q8IM66}",0.796937,4,0.397808,True,True,5,0.750,True,True
31,31,"{Q8IL48, C6KTA3, A0A5K1K967, C6KSV2}",0.795290,4,0.392043,True,True,5,0.500,True,True
15,15,"{Q8IEQ1, Q8IAR6, Q8IKH3, Q8IJM0, Q8ILE3, Q8IC0...",0.758082,8,0.392335,True,True,5,0.625,True,True
32,32,"{C6S3G2, O96252, Q8I3N3, C6KT09}",0.747036,4,0.393874,True,True,5,0.750,True,True
40,40,"{C0H4W2, Q8I3Q7, Q8IIT3, Q8I5V6}",0.701087,4,0.413668,True,True,5,0.250,True,True


In [809]:
# convert csv to tsv
s3_ppi_clusters = pd.read_csv("data/Stage_3_PPI_predicted_features_with_clusters1.csv", sep=",")
s3_ppi_clusters.to_csv("data/Stage_3_PPI_predicted_features_with_clusters1.tsv", sep="\t", index=False)

s5_ppi_clusters = pd.read_csv("data/Stage_5_PPI_predicted_features_with_clusters1.csv", sep=",")
s5_ppi_clusters.to_csv("data/Stage_5_PPI_predicted_features_with_clusters1.tsv", sep="\t", index=False)

# create a map of protein accessions to protein name/functions
protein_desc_map = {}
for row in s5_ppi_clusters.iterrows():
    if row[1]["Protein1"] not in protein_desc_map:
        protein_desc_map[row[1]["Protein1"]] = row[1]["Description1"]
    if row[1]["Protein2"] not in protein_desc_map:
        protein_desc_map[row[1]["Protein2"]] = row[1]["Description2"]
        
for row in s3_ppi_clusters.iterrows():
    if row[1]["Protein1"] not in protein_desc_map:
        protein_desc_map[row[1]["Protein1"]] = row[1]["Description1"]
    if row[1]["Protein2"] not in protein_desc_map:
        protein_desc_map[row[1]["Protein2"]] = row[1]["Description2"]

def get_protein_desc_from_acccession(cluster):
    return [protein_desc_map[protein] for protein in cluster]


In [810]:
get_protein_desc_from_acccession(s5_df["s5_proteins"][0])
get_protein_desc_from_acccession(s3_df["s3_proteins"][4])

['Triosephosphate isomerase',
 '20 kDa chaperonin',
 'Heat shock protein 70',
 'Aminopeptidase P']

In [811]:
# append putative tag if cluster contains a putative protein
def append_putative(row, protein_col):
    proteins = get_protein_desc_from_acccession(row[protein_col])
    contains_putative = False
    for p in proteins:
        if 'putative' in p:
            contains_putative = True
            break
    return contains_putative

def append_uncharacterized(row, protein_col):
    proteins = get_protein_desc_from_acccession(row[protein_col])
    contains_uncharacterized = False
    for p in proteins:
        if 'uncharacterized' in p.lower():
            contains_uncharacterized = True
            break
    return contains_uncharacterized

s5_df["contains_putative"] = s5_df.apply(append_putative, args=["s5_proteins"], axis=1)
s3_df["contains_putative"] = s3_df.apply(append_putative, args=["s3_proteins"], axis=1)

s5_df["contains_uncharacterized"] = s5_df.apply(append_uncharacterized, args=["s5_proteins"], axis=1)
s3_df["contains_uncharacterized"] = s3_df.apply(append_uncharacterized, args=["s3_proteins"], axis=1)

In [812]:
# rename columns
cluster_concat_df = pd.concat(
    [s3_df.rename(columns={"s3_number": "Number",
                      "s3_proteins": "Proteins",
                      "s3_avg_spearman": "avg_spearman",
                      "s3_size": "size",
                      "s3_essentiality": "essentiality"}, inplace=False),
    s5_df.rename(columns={"s5_number": "Number",
                      "s5_proteins": "Proteins",
                      "s5_avg_spearman": "avg_spearman",
                      "s5_size": "size",
                      "s5_essentiality": "essentiality"}, inplace=False)]
)
cluster_concat_df = cluster_concat_df[cluster_concat_df["size"] > 2]
cluster_concat_df["descriptions"] = cluster_concat_df["Proteins"].apply(get_protein_desc_from_acccession)

In [813]:
cluster_concat_df[cluster_concat_df["contains_uncharacterized"]].sort_values("essentiality", ascending=False).head(5)

,Number,Proteins,avg_spearman,size,avg_spearman_random,significant,large,stage,essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized,descriptions
74,74,"{Q8IKS0, Q8IM24, Q8I3X7, Q8IKW0}",0.560441,4,0.390596,True,True,5,1.000000,True,False,False,True,"[LCCL domain-containing protein, LCCL/lectin a..."
68,68,"{Q8IKA6, Q76NN6, Q7KQK6, A0A144A0F5}",0.569499,4,0.373611,True,True,5,1.000000,False,False,True,True,"[DnaJ protein, putative, Ran-specific GTPase-a..."
56,56,"{Q8IKC8, Q8IEB8, Q8IK07, Q8I485}",0.091733,4,0.392239,False,True,5,1.000000,False,True,False,True,"[Exported protein 2, Uncharacterized protein, ..."
0,0,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...",0.376935,16,0.366112,True,True,3,0.875000,False,False,True,True,"[40S ribosomal protein S7, Uncharacterized pro..."
10,10,"{C6KSV8, Q8I2X3, C0H571, Q8I484, Q8I5J2, Q8IBN4}",0.421278,6,0.395242,True,True,5,0.833333,False,False,True,True,"[ATP synthase-associated protein, putative, Gl..."


## Functions for analysis

#### For Clusters

In [814]:
# By clusters
def get_clusters_for_protein(protein, cluster_concat_df=cluster_concat_df):
    """gets a clusters by protein

    Args:
        protein (str): protein ID, either uniprot accession or plasmoDB ID
        cluster_concat_df (_type_, optional): DF containing clusters by row. Defaults to cluster_concat_df.

    Returns:
        _type_: augmented dataframe with clusters containing protein
    """
    print("get clusters for protein with function: ", protein_desc_map[protein])
    # get all clusters for a given protein
    clusters = cluster_concat_df[cluster_concat_df["Proteins"].apply(lambda x: protein in x)]
    return clusters

In [815]:
get_clusters_for_protein("Q8I0V2", cluster_concat_df)

get clusters for protein with function:  ATP synthase subunit beta


,Number,Proteins,avg_spearman,size,avg_spearman_random,significant,large,stage,essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized,descriptions
0,0,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...",0.376935,16,0.366112,True,True,3,0.875000,False,False,True,True,"[40S ribosomal protein S7, Uncharacterized pro..."
2,2,"{Q8IET7, Q8I0P6, O96252, O97247, K7NTP5, Q8IB0...",0.398492,26,0.368785,True,True,3,0.884615,False,False,True,False,"[40S ribosomal protein S7, Elongation factor 1..."
14,14,"{Q8IBV7, Q8IAX8, C6KSV0, Q8I608, K7NTP5, Q8IK8...",0.331886,14,0.369976,False,True,3,0.928571,False,False,True,False,"[Histone H2B, DNA/RNA-binding protein ALBA1, H..."
18,18,"{Q8IET7, Q8I0P6, O96252, O97247, Q8IK89, K7NTP...",0.426576,21,0.352300,True,True,3,0.904762,False,False,True,False,"[40S ribosomal protein S7, Elongation factor 1..."
24,24,"{Q8IKG9, Q8IBD1, K7NTP5, O96252, Q8IK89, Q76NN...",0.312215,13,0.355444,False,True,3,0.692308,False,False,True,True,"[Uncharacterized protein, Sporozoite and liver..."
2,2,"{Q8IET7, Q8IBV7, C0H5C2, K7NTP5, Q8I3R0, Q8I0P...",0.516712,65,0.394348,True,True,5,0.753846,False,False,True,False,"[40S ribosomal protein S7, Histone H2B, 40S ri..."
59,59,"{Q8I2X3, A0A5K1K8W5, Q8IM15, Q8IBN4, Q8I0V2, A...",0.287220,6,0.411393,False,True,5,0.833333,True,False,True,False,"[Glideosome-associated protein 50, Secreted oo..."


#### For Proteins

In [816]:
# By proteins
def get_essential_proteins(protein_concat_df=protein_concat_df):
    return protein_concat_df[protein_concat_df["essential"] == True]

In [817]:
essential = get_essential_proteins()
essential[(essential["Degree"] < 10) & (essential["Degree"] > 5)].head()

,Unnamed: 0,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
4,4,Q8IIA4,PF3D7_1126000,threonine--tRNA ligase,0.119,-2.949,"['Q8IBS3', 'Q8IE10', 'Q8IDK7', 'Q8IDZ9', 'Q8IL...",True,7,0.008290,0.312109,5,False,True
6,6,C0H571,PF3D7_0929400,high molecular weight rhoptry protein 2,0.119,-3.010,"['Q8IKC8', 'Q8IBN4', 'Q6ZMA7', 'Q8I4T3', 'C6KS...",True,9,0.002461,0.310253,5,True,True
21,21,Q8IKF0,PF3D7_1468700,eukaryotic initiation factor 4A,0.120,-2.990,"['Q8I0V4', 'Q9TY94', 'C6KT23', 'Q8IDB0', 'Q8IC...",True,7,0.000639,0.316163,5,True,True
25,25,Q8I3I6,PF3D7_0528100,"AP-1/2 complex subunit beta, putative",0.120,-2.959,"['Q8IB24', 'Q8ILG6', 'Q8I2X4', 'C0H5H0', 'Q8I3...",True,6,0.005161,0.338432,5,False,True
28,28,Q8IIJ6,PF3D7_1117100,ubiquitin carboxyl-terminal hydrolase UCH54,0.121,-2.890,"['Q8II71', 'C0H5H0', 'Q8IJW0', 'A0A5K1K9F3', '...",True,9,0.000112,0.339060,5,False,True


In [818]:
# get cluster pairs with only one common protein
crossed_df[(crossed_df["num_overlap"] == 1) & (crossed_df["s3_size"] > 2) & (crossed_df["s5_size"] > 2)].head()

,s3_number,s5_number,s3_proteins,s5_proteins,s3_avg_spearman,s5_avg_spearman,s3_size,s5_size,total_size,common_proteins,num_overlap,percent_overlap,jaccard_index,similarity_score
8,0,8,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8I0V4, A0A5K1K910, Q8IKF0, Q8III5, Q7KQL5, Q...",0.376935,0.489262,16,6,22,[Q8IKF0],1,0.166667,0.047619,0.107143
14,0,14,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8IJ28, Q8IKJ0, Q8IBV7, Q8IJM9}",0.376935,0.503623,16,4,20,[Q8IBV7],1,0.250000,0.052632,0.151316
20,0,20,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8ILZ7, Q8IJX3, Q8IK89, O97306}",0.376935,0.387022,16,4,20,[Q8IK89],1,0.250000,0.052632,0.151316
33,0,33,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8I1S0, Q8I5B3, O96221, Q8I5L6, Q8IB60, Q8ILX1}",0.376935,0.496311,16,6,22,[Q8I5L6],1,0.166667,0.047619,0.107143
59,0,59,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...","{Q8I2X3, A0A5K1K8W5, Q8IM15, Q8IBN4, Q8I0V2, A...",0.376935,0.287220,16,6,22,[Q8I0V2],1,0.166667,0.047619,0.107143


In [819]:
tmp = get_clusters_for_protein("Q8IKF0")
tmp[tmp["contains_putative"]]

get clusters for protein with function:  RNA helicase


,Number,Proteins,avg_spearman,size,avg_spearman_random,significant,large,stage,essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized,descriptions
0,0,"{Q8IET7, Q8IKG9, Q8IBV7, C6KSV0, Q8I0P6, K7NTP...",0.376935,16,0.366112,True,True,3,0.875000,False,False,True,True,"[40S ribosomal protein S7, Uncharacterized pro..."
2,2,"{Q8IET7, Q8I0P6, O96252, O97247, K7NTP5, Q8IB0...",0.398492,26,0.368785,True,True,3,0.884615,False,False,True,False,"[40S ribosomal protein S7, Elongation factor 1..."
10,10,"{Q8IK89, Q8I0V4, Q8II24, Q8IDZ8, Q8IKF0, C0H4V...",0.638065,9,0.375679,True,True,3,0.888889,False,False,True,False,"[Trailer hitch homolog, putative, Endoplasmin,..."
18,18,"{Q8IET7, Q8I0P6, O96252, O97247, Q8IK89, K7NTP...",0.426576,21,0.352300,True,True,3,0.904762,False,False,True,False,"[40S ribosomal protein S7, Elongation factor 1..."
24,24,"{Q8IKG9, Q8IBD1, K7NTP5, O96252, Q8IK89, Q76NN...",0.312215,13,0.355444,False,True,3,0.692308,False,False,True,True,"[Uncharacterized protein, Sporozoite and liver..."
8,8,"{Q8I0V4, A0A5K1K910, Q8IKF0, Q8III5, Q7KQL5, Q...",0.489262,6,0.401733,True,True,5,0.500000,False,False,True,False,"[Endoplasmin, putative, aspartate carbamoyltra..."
77,77,"{Q8IBG1, Q8I0V4, A0A144A1R5, Q8IC05, Q8IKF0, Q...",0.302591,9,0.419843,False,True,5,0.777778,False,False,True,False,"[Dynein heavy chain-like protein 1, Endoplasmi..."


In [820]:
cluster_concat_df[cluster_concat_df["contains_putative"] & cluster_concat_df["unique between stages"] & (cluster_concat_df["stage"] == 5)].sort_values(["avg_spearman", "essentiality"], ascending=False).head(5)

,Number,Proteins,avg_spearman,size,avg_spearman_random,significant,large,stage,essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized,descriptions
66,66,"{Q8II42, Q8I3A1, P61074, Q8II92, C6S3I6}",0.940508,5,0.399015,True,True,5,0.600000,True,False,True,False,"[Nucleic acid binding protein, putative, Repli..."
30,30,"{C0H4C7, Q8IBR6, Q8I3Y6, Q8ILS7, Q8IBZ4, Q8II8...",0.879400,7,0.397545,True,True,5,0.428571,True,False,True,True,[Mediator of RNA polymerase II transcription s...
36,36,"{C6S3I6, Q8II42, Q8I3A1, Q8I5Q3}",0.838632,4,0.423284,True,True,5,0.500000,True,False,True,False,"[Replication factor A protein 3, putative, Nuc..."
19,19,"{Q8I1S0, A0A143ZY58, Q8IKH3, Q8IM66}",0.796937,4,0.397808,True,True,5,0.750000,True,True,True,False,"[Small GTP-binding protein sar1, ADP-ribosylat..."
31,31,"{Q8IL48, C6KTA3, A0A5K1K967, C6KSV2}",0.795290,4,0.392043,True,True,5,0.500000,True,True,True,False,"[tRNA import protein tRIP, ATPase, Elongation ..."


In [821]:
cluster_concat_df[cluster_concat_df["contains_putative"] & cluster_concat_df["unique between stages"] & (cluster_concat_df["stage"] == 5)].sort_values(["avg_spearman", "essentiality"], ascending=False).head(5)

,Number,Proteins,avg_spearman,size,avg_spearman_random,significant,large,stage,essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized,descriptions
66,66,"{Q8II42, Q8I3A1, P61074, Q8II92, C6S3I6}",0.940508,5,0.399015,True,True,5,0.600000,True,False,True,False,"[Nucleic acid binding protein, putative, Repli..."
30,30,"{C0H4C7, Q8IBR6, Q8I3Y6, Q8ILS7, Q8IBZ4, Q8II8...",0.879400,7,0.397545,True,True,5,0.428571,True,False,True,True,[Mediator of RNA polymerase II transcription s...
36,36,"{C6S3I6, Q8II42, Q8I3A1, Q8I5Q3}",0.838632,4,0.423284,True,True,5,0.500000,True,False,True,False,"[Replication factor A protein 3, putative, Nuc..."
19,19,"{Q8I1S0, A0A143ZY58, Q8IKH3, Q8IM66}",0.796937,4,0.397808,True,True,5,0.750000,True,True,True,False,"[Small GTP-binding protein sar1, ADP-ribosylat..."
31,31,"{Q8IL48, C6KTA3, A0A5K1K967, C6KSV2}",0.795290,4,0.392043,True,True,5,0.500000,True,True,True,False,"[tRNA import protein tRIP, ATPase, Elongation ..."


In [822]:
protein_concat_df.sort_values("Degree", ascending=False).head(5)

,Unnamed: 0,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
114,114,C0H5H0,PF3D7_1344200,endoplasmic reticulum chaperone GRP170,0.143,-2.839,"['O97282', 'Q8II43', 'Q8IFP3', 'Q8ILP6', 'C6KS...",True,69,0.148560,0.445327,5,True,True
109,109,Q8IAX5,PF3D7_0813900,"40S ribosomal protein S16, putative",0.141,-3.879,"['Q8IE10', 'C6KT19', 'C6KT25', 'Q8IIB4', 'Q8IE...",True,58,0.033885,0.426100,5,False,True
337,341,Q8IM10,PF3D7_1408600,"40S ribosomal protein S8e, putative",1.000,-2.487,"['Q8IL02', 'O97313', 'Q8I487', 'Q8I0P6', 'Q8II...",False,56,0.013215,0.416858,5,False,False
41,41,Q8I323,PF3D7_0912900,"26S proteasome regulatory subunit RPN8, putative",0.122,-3.009,"['Q8ILE3', 'Q76NM6', 'Q8IKB2', 'Q8I0V2', 'Q8I2...",True,53,0.053808,0.421672,5,False,True
62,62,Q8IDR9,PF3D7_1342000,40S ribosomal protein S6,0.127,-2.986,"['Q8ILI2', 'Q8IIA2', 'Q8IK15', 'Q8IIV2', 'A0A1...",True,53,0.022999,0.415909,5,False,True


## Improving Clusters with Coexpression

In [823]:
# Checking for clusters with size < 6
s3_df[s3_df["s3_size"] < 6].sort_values("s3_size", ascending=False).head()

,s3_number,s3_proteins,s3_avg_spearman,s3_size,avg_spearman_random,significant,large,stage,s3_essentiality,unique between stages,unique within stage,contains_putative,contains_uncharacterized
9,9,"{Q8I0X1, Q8ILB6, P61074, Q8IEJ6, O97227}",0.678557,5,0.365600,True,True,3,1.00,False,False,True,False
11,11,"{Q8IL75, Q8IKI8, Q8I3N3, Q8I2N1, Q8I2I2}",0.704249,5,0.376922,True,True,3,0.60,False,True,True,False
4,4,"{Q7KQM0, Q8IDZ8, Q8II24, A0A144A2H0}",0.935112,4,0.346397,True,True,3,0.75,True,False,False,False
5,5,"{Q8IEC8, Q8IDN6, Q8ILV6, Q8I2W2}",0.634717,4,0.343030,True,True,3,0.75,False,True,True,False
17,17,"{Q8IHR8, Q76NM4, A0A5K1K8H7, Q8I3W9}",0.486660,4,0.358231,True,True,3,1.00,False,True,False,False


In [824]:
s3_pairs_correlation = pd.read_csv("data/generated_tables/s3_interacting_pairs_correlation.tsv", sep="\t")
s5_pairs_correlation = pd.read_csv("data/generated_tables/s5_interacting_pairs_correlation.tsv", sep="\t")

In [825]:
from itertools import combinations

x = combinations(s3_df[s3_df["s3_size"] < 6].sort_values("s3_size", ascending=False).head(1)["s3_proteins"].values[0], 2)

In [826]:
s3_essentiality_df.dropna(subset=["3D7_MIS"], inplace=True)
s3_essentiality_df

,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
0,C6KST3,PF3D7_0608500,"proteasome subunit alpha type-2, putative",0.119,-3.100,"['Q6ZMA8', 'Q8I0U7', 'Q8II43', 'Q8I2X4', 'C6KT...",True,18,0.002969,0.234283,3,False,True
1,Q8IKJ0,PF3D7_1464700,"ATP synthase (C/AC39) subunit, putative",0.119,-3.121,"['Q8I2X4', 'O96252', 'Q6ZMA8', 'K7NTP5', 'Q76N...",True,5,0.027947,0.311850,3,False,True
2,C0H571,PF3D7_0929400,high molecular weight rhoptry protein 2,0.119,-3.010,"['Q8IKC8', 'O77310', 'Q8I395', 'Q8IBN4', 'Q8IL...",True,7,0.000554,0.049310,3,False,True
3,Q8IIJ8,PF3D7_1116800,heat shock protein 101,0.119,-3.014,"['Q8IB24', 'Q8IB03', 'O96252', 'Q8ILA1', 'K7NT...",True,9,0.004823,0.282287,3,False,True
4,A0A143ZZK9,PF3D7_1211900,non-SERCA-type Ca2+ -transporting P-ATPase,0.119,-3.089,['Q8IB24'],True,1,0.000000,0.239760,3,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Q8IFM9,PF3D7_0423700,early transcribed membrane protein 4,1.000,0.000,['Q8IJM9'],False,1,0.000000,0.006410,3,False,False
152,Q8IEA6,PF3D7_1320800,dihydrolipoyllysine-residue succinyltransferas...,1.000,0.398,"['Q8I6S5', 'Q8IJ34']",False,2,0.009843,0.203321,3,False,False
153,Q8IES0,PF3D7_1304500,"small heat shock protein, putative",1.000,-2.102,"['K7NTP5', 'Q8I0V2', 'Q8IJW0']",False,3,0.000000,0.261495,3,False,False
154,Q8IDG9,PF3D7_1353100,"Plasmodium exported protein, unknown function",1.000,-0.310,"['Q8IEI6', 'Q8I490']",False,2,0.000000,0.020979,3,False,False


In [827]:
cluster_9_pairs = []
for y in x:
    cluster_9_pairs.append(y)

In [828]:
cluster_9_pairs

[('Q8I0X1', 'Q8ILB6'),
 ('Q8I0X1', 'P61074'),
 ('Q8I0X1', 'Q8IEJ6'),
 ('Q8I0X1', 'O97227'),
 ('Q8ILB6', 'P61074'),
 ('Q8ILB6', 'Q8IEJ6'),
 ('Q8ILB6', 'O97227'),
 ('P61074', 'Q8IEJ6'),
 ('P61074', 'O97227'),
 ('Q8IEJ6', 'O97227')]

In [829]:
s5_essentiality_df[s5_essentiality_df["Accession ID"] == "C0H5H0"]

,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
114,C0H5H0,PF3D7_1344200,endoplasmic reticulum chaperone GRP170,0.143,-2.839,"['O97282', 'Q8II43', 'Q8IFP3', 'Q8ILP6', 'C6KS...",True,69,0.14856,0.445327,5,False,True


In [830]:
s3_essentiality_df.sort_values("Betweenness Centrality", ascending=False).head(5)

,Accession ID,Gene ID,Product Description,3D7_MIS,3D7_MFS,Neighbours,essential,Degree,Betweenness Centrality,Closeness Centrality,stage,redundant,under_MIS_threshold
146,K7NTP5,PF3D7_0831700,heat shock protein 70,1.000,0.585,"['Q8IJN7', 'C6KSV0', 'Q76NN8', 'Q8IB24', 'Q8IE...",True,41,0.122628,0.373715,3,True,False
104,C0H4V6,PF3D7_0818200,14-3-3 protein,0.585,-3.082,"['Q7KQL9', 'P61074', 'Q8I0V4', 'Q8IJN7', 'Q8II...",False,21,0.086894,0.348331,3,False,False
62,Q8II24,PF3D7_1134000,heat shock protein 70,0.182,-2.879,"['C0H4V6', 'Q8IKK7', 'Q8IKF0', 'O97227', 'Q8I2...",True,14,0.082584,0.332042,3,True,True
59,C6KSV0,PF3D7_0610400,histone H3,0.176,-2.610,"['K7NTP5', 'C6KT18', 'Q8IJ34', 'Q8IIV1', 'Q8IL...",True,19,0.059210,0.313972,3,False,True
36,Q8IC01,PF3D7_0708800,heat shock protein 110,0.140,-2.848,"['Q6ZMA8', 'Q8IDG2', 'Q8IK90', 'Q8I2J3', 'Q8ID...",True,21,0.057584,0.292113,3,False,True
